In [11]:
import tables
import glob
import numpy as np
from multiprocessing import Pool

In [12]:
hfile = tables.open_file("Data/training/train2.h5", "r")

In [13]:
htable = np.fromfile("injunction_table_lst.pny", offset=8, dtype=np.uint16)
def get_img_matrix(img, nb_img):
    matrix = np.zeros(55*55)

    mat = np.zeros((nb_img,55,55),dtype=np.float32)

    for i in range(nb_img):
        matrix[htable] = img[i]
        res = matrix.reshape(55,55)
        mat[i] = res

    return mat

In [14]:
#return the singular values
def svd(res):
    # Applying SVD
    U, S, VT = np.linalg.svd(res,full_matrices=False,# It's not necessary to compute the full matrix of U or V
        compute_uv=True) # Deterministic SVD

    S = np.diag(S) #singular value matrix
   
    return S

In [19]:
def call_svd(file_offset):
    file, outputsvd = file_offset
    hfile =tables.open_file(file,"r")
    sv= hfile.root.Shower.gerbe.col("sv")
    

    outputsvd[:] = sv

In [22]:
train_fnames = glob.glob('hash_training/*.h5',recursive=True) #500 files
nbevent = 0
file_offset = []
file_view = []

for file in train_fnames:
    hfile = tables.open_file(file, "r")
    nrows = hfile.root.Shower.gerbe.nrows
    file_offset.append((file,nbevent, nbevent+nrows))
    nbevent = nbevent + nrows
    hfile.close()

tab_event_svd =  np.zeros((nbevent,55))
for tab in file_offset:
    file_view.append((tab[0], tab_event_svd[tab[1]:tab[2]]))
with Pool(96) as p:
        p.map(call_svd, file_view)